# Introduction

See "demo.ipynb" notebook for a general introduction.

This notebook demonstrates how the fabric model "specfab" (https://nicholasmr.github.io/specfab) can be used to construct a vertical profile of the $c$-axis Orientation Distribution Function (ODF) relevant for domes or ridges, assuming a depth-constant strain-rate.
For non-constant strain-rate histories, see https://nicholasmr.github.io/specfab/cpo-dynamics-tranisotropic/

The profile can then, in turn, be fed to the radar model.

In [ ]:
import numpy as np
from specfabpy import specfab as sf
from specfabpy import integrator as sfint
import matplotlib.pyplot as plt
yr2s = 31556952 

### Site config

H       = 2000 # Ice column thickness
ezz_obs = 5e-5 # Vertical strain rate at dome or ridge (yr^-1)
r       = 1    # Horizontal extension assymmetry: r=0 for dome (unconfined pure shear), r=+/-1 for perfect x- or y-aligned ridge (confined pure shear)

### Fabric model 

L       = 12    # Spectral truncation of fabric model
Nt      = 200   # Number of integration time steps for fabric model
axis    = 2     # Compression along z-axis
ezz_end = -0.95 # Simulate parcel deformation until this strain_zz target
T       = 1/ezz_obs * yr2s # Deformation time scale, see https://nicholasmr.github.io/specfab/deformation-modes

### Model parcel evolution along vertical column

# See https://nicholasmr.github.io/specfab/Lagrangian-parcel-integrator

lm, nlm_len = sf.init(L)

# Initial fabric state of parcel, prescribed in terms of a^(2) eigenvalues
nlm0 = np.zeros((nlm_len), dtype=np.complex64)
#l1, l2 = 0.2, 0.2 # Horizontal eigenvalues for single maximum
l1, l2 = 0.2, 0.35 # Horizontal eigenvalues for girdle
nlm0[:sf.L2len] = sf.a2_to_nlm(np.diag([l1,l2,1-l1-l2])) # Near surface fabric

mod = dict(type='ps', r=r, axis=axis, T=T) # Mode of deformation (ps=pure shear)
fabparams = dict(iota=1, nu=1, Lambda=None, Gamma0=None) # Lattice rotation only, no DRX, default regularization
nlm_sf, F, time, ugrad = sfint.lagrangianparcel(sf, mod, ezz_end, Nt=Nt, nlm0=nlm0, **fabparams)

eij = np.array([sf.F_to_strain(F[n,:,:]) for n in np.arange(Nt)]) # Strain tensor from deformation gradient F
depth = H*eij[:,axis,axis] # Depth axis
ageka = 1e-3*time[:-1]/yr2s # Age at depth

### Interpolate fabric profile onto equidistant z-grid for radar model

from scipy import interpolate

N_layers = 100  # Number of layers (for radar model below)

nlm = np.zeros((N_layers, nlm_len), dtype=np.complex64) # Gridded expansion coefficients
z = np.linspace(0, depth[-1], N_layers) # Equidistant z-grid 
ageka_sf = interpolate.interp1d(depth, ageka)(z) # Corresponding age at depth

for ii in np.arange(nlm_len): nlm[:,ii] = interpolate.interp1d(depth, nlm_sf[:-1,ii])(z)
nlm[0,1:] = 0 # Ensure the top layer is isotropic (required by model)


### Plot eigenvalues

lami = np.zeros((Nt, 3)) # Eigenvalues 
for n in np.arange(Nt): _,_,_, lami[n,:] = sf.frame(nlm_sf[n,:], 'e') # "e" = eigenframe 

fig, (ax1, ax2) = plt.subplots(1, 2)

for ii in range(3): ax1.plot(lami[:,ii], depth[:], '.', label=r'$\lambda_%i$'%(1+ii))
ax1.set_ylabel(r'$z$ (m)')
ax1.set_xlabel(r'$\lambda_i$')
ax1.legend(loc=1)

ax2.plot(lami[:,2]-lami[:,1], depth[:], '.', label=r'$\lambda_3-\lambda_2$')
ax2.set_ylabel(r'$z$ (m)')
ax2.set_xlabel(r'$\lambda_3 - \lambda_2$')

def forward(x): return np.interp(x, depth, ageka)
def inverse(x): return np.interp(x, ageka, depth)
secax1 = ax1.secondary_yaxis('right', functions=(forward, inverse))
secax2 = ax2.secondary_yaxis('right', functions=(forward, inverse))
secax1.set_ylabel(r'Age (ka)')
secax2.set_ylabel(r'Age (ka)')

fig.tight_layout()

# Calculate radar returns

To calculate the radar returns 

* $\bar{P}_{jk}$          : Mean return power for Tx and Rx polarizations of $j=\lbrace \mathrm{H,V}\rbrace$ and $k=\lbrace \mathrm{H,V}\rbrace$, respectively
* $\delta{P}_{jk}$        : Angular power anomaly for Tx and Rx polarizations of $j=\lbrace \mathrm{H,V}\rbrace$ and $k=\lbrace \mathrm{H,V}\rbrace$, respectively
* $\varphi_{\mathrm{HV}}$ : HH$-$VV Covariance phase angle

given the above fabric profile, run:


In [ ]:
import sys
sys.path.insert(0, '../../lib') # Add model library path
from layerstack import *
from plottools import *

### Setup layer stack

N_frames = 50           # Number of horizontally rotated frames (beta) between 0 and Pi 
epsc     = 3.17         # Single grain permittivity parallel to c-axis
epsa     = epsc - 0.034 # Single grain permittivity perpendicular to c-axis
sigma    = 1e-5         # Isotropic macroscopic conductivity

lstack = LayerStack(nlm, z, N_frames=N_frames, epsa=[epsa], epsc=[epsc], sigma=[sigma]) # Assumes constant epsa, epsc and sigma throughout the column if len()==1

### Transmitted plane wave

E0       = 1e3   # Tx E-field amplitude
f        = 300e6 # 179e6 # Tx frequency in Hz
alpha    = np.deg2rad(0) # Angle of incidence in radians (0 = normal incidence)

returns = lstack.get_returns(E0, f=f, alpha=alpha) # returns = (Pm_HH,Pm_HV, dP_HH,dP_HV, c_HHVV, E_HH,E_HV)

### Plot results

(eigvals, e1,e2,e3, a2) = lstack.get_eigenbasis()

(plt, *_) = plot_returns(lstack.z, returns, a2, eigvals) # NOTE: a2 contains only information about the lowest order harmonic modes. The ODF plots will therefore NOT look like the above plotted ODFs.
plt.show()